### Prompts

Prompt allow you to create System Messages with input variables, for example, this: 
SystemMessage(content="You are a helpful assistant that translates the English to Spanish.") 
English and Spanish may be dynamic and this can be achieved with templates

In [7]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [8]:
TEMPLATE = """
You are a helpful assistant that translates the {input_language} text to {output_language}.
"""

In [9]:
from langchain.prompts import PromptTemplate
from langchain.schema import prompt_template


prompt_template = PromptTemplate.from_template(
    template=TEMPLATE,
)
prompt_template.format(input_language="English", output_language="French")

'\nYou are a helpful assistant that translates the English text to French.\n'

Passing the input_variables to the constructor will provide additional validation for the template

In [10]:
prompt_template = PromptTemplate(
    template=TEMPLATE, input_variables=["input_language", "output_language"]
)
prompt_template.format(input_language="French", output_language="English")

'\nYou are a helpful assistant that translates the French text to English.\n'

### Few Shot Prompt: Provide few examples in the template

In [11]:
TEMPLATE = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with following keys:
sentiment
subject

text: {input}
"""

To improve performance we can provide examples to increase the quality of the output

In [12]:
TEMPLATE = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with following keys:
sentiment
subject

text: {input}

Examples:
text: The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the service is impeccable.
sentiment: positive
subject: BellaVista

text: BellaVista restaurant was alright. The food was decent but nothing special.
sentiment: neutral
subject: BellaVista

text: I was disappointed by the BellaVista restaurant. The food was bland and the service was slow.
sentiment: negative
subject: BellaVista
"""

In [13]:
prompt_template = PromptTemplate(template=TEMPLATE, input_variables=["input"])
prompt_template.format(
    input="The food at BellaVista just blew my mind. The flavors are rich and the service is impeccable."
)

'\nInterprete the text and evaluate the text.\nsentiment: is the text in a positive, neutral or negative sentiment?\nsubject: What subject is the text about? Use exactly one word.\n\nFormat the output as JSON with following keys:\nsentiment\nsubject\n\ntext: The food at BellaVista just blew my mind. The flavors are rich and the service is impeccable.\n\nExamples:\ntext: The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the service is impeccable.\nsentiment: positive\nsubject: BellaVista\n\ntext: BellaVista restaurant was alright. The food was decent but nothing special.\nsentiment: neutral\nsubject: BellaVista\n\ntext: I was disappointed by the BellaVista restaurant. The food was bland and the service was slow.\nsentiment: negative\nsubject: BellaVista\n'

LangChain also provides a FewShotPromptTemplate class which allows creating the examples more modularized

In [14]:
from langchain.prompts import FewShotPromptTemplate

examples = [
    {
        "text": "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the service is impeccable.",
        "response": "sentiment: positive\nsubject: BellaVista",
    },
    {
        "text": "BellaVista restaurant was alright. The food was decent but nothing special.",
        "response": "sentiment: neutral\nsubject: BellaVista",
    },
    {
        "text": "I was disappointed by the BellaVista restaurant. The food was bland and the service was slow.",
        "response": "sentiment: negative\nsubject: BellaVista",
    },
]

In [15]:
example_prompt = PromptTemplate(
    template="Text: {text}\n{response}", input_variables=["input", "response"]
)

In [16]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="text: {input}",
    input_variables=["input"],
)

In [17]:
print(
    prompt.format(
        input="The food at BellaVista just blew my mind. The flavors are rich and the service is impeccable."
    )
)

Text: The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the service is impeccable.
sentiment: positive
subject: BellaVista

Text: BellaVista restaurant was alright. The food was decent but nothing special.
sentiment: neutral
subject: BellaVista

Text: I was disappointed by the BellaVista restaurant. The food was bland and the service was slow.
sentiment: negative
subject: BellaVista

text: The food at BellaVista just blew my mind. The flavors are rich and the service is impeccable.


### Chain of thought prompting

Instead of just providing examples, we can also provide examples which include the whole thought process of why a review is negative, neutral or positive

In [18]:
TEMPLATE = """
Interprete the text and evaluate the text. Determine if the text has a positive, neutral or negative sentiment.
Also Format the output as JSON with the following keys:
sentiment
subject

text: {input}

Chain-of-Thought Prompts:
Let's start by evaluating a statement. Consider: "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the service is impeccable."
It sounds like a positive review for BellaVista

Based on the postive nature of the statement, how would you format your response?
{"sentiment": "positive", "subject": "BellaVista"}

Now think about this: "SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too seet for my taste"
Does it sounds positive? Not particularly, it seems like a mix of good and not so good elements so its nautral

Given the neutral sentiment how should this be presented?
{"sentiment": "neutral", "subject": "SeoulSavor"}

Laastly, ponder on this: "I was let down by MunichMeals. The potato salad lacked flavor and the schnitzel was overcooked."
The statement is clearly negative, how would you format the response?
{"sentiment": "negative", "subject": "MunichMeals"}

"""

### Composition

You can also compose multiple prompts together. This can be especially useful if you want to reuse parts of prompts

In [19]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

# Introduction
introduction_template = """
Interprete the text and evaluate the text. Determine if the text has a positive, neutral, or negative sentiment. Also, identify the subject of the text in one word.
"""
introduction_prompt = PromptTemplate.from_template(introduction_template)

# Example
example_template = """
Chain-of-Thought Prompts:
Let's start by evaluating a statement. Consider: "{example_text}". How does this make you feel about {example_subject}?
Response: {example_evaluation}

Based on the {example_sentiment} nature of that statement, how would you format your response?
Response: {example_format}
"""
example_prompt = PromptTemplate.from_template(example_template)

# Execution
execution_template = """
Now, execute this process for the text: "{input}".
"""
execution_prompt = PromptTemplate.from_template(execution_template)

# Composing the full prompt
full_template = """{introduction}

{example}

{execution}"""
full_prompt = PromptTemplate.from_template(full_template)

# PipelinePrompts
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("execution", execution_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

In [20]:
print(
    pipeline_prompt.format(
        example_text="The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.",
        example_subject="BellaVista",
        example_evaluation="It sounds like a positive review for BellaVista.",
        example_sentiment="positive",
        example_format='{ "sentiment": "positive", "subject": "BellaVista" }',
        input="The new restaurant downtown has bland dishes and the wait time is too long.",
    )
)


Interprete the text and evaluate the text. Determine if the text has a positive, neutral, or negative sentiment. Also, identify the subject of the text in one word.



Chain-of-Thought Prompts:
Let's start by evaluating a statement. Consider: "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.". How does this make you feel about BellaVista?
Response: It sounds like a positive review for BellaVista.

Based on the positive nature of that statement, how would you format your response?
Response: { "sentiment": "positive", "subject": "BellaVista" }



Now, execute this process for the text: "The new restaurant downtown has bland dishes and the wait time is too long.".



### Serializing prompts

Example is an easy prompt since serializing does not work for the PipelinePromptTemplate (yet)

In [21]:
prompt = PromptTemplate(
    input_variables=["input"], template="Tell me a joke about {input}"
)
prompt.save("prompt.yaml")
prompt.save("prompt.json")

In [22]:
from langchain.prompts import load_prompt

prompt = load_prompt("prompt.yaml")
prompt.format(input="chickens")

'Tell me a joke about chickens'

In [23]:
prompt = load_prompt("prompt.json")
prompt.format(input="cows")

'Tell me a joke about cows'